<a href="https://colab.research.google.com/github/hfelizzola/Investigaciones-de-Operaciones-I/blob/main/programacion-lineal/02_analisis_sensibilidad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análisis de Sensibilidad en Programación Lineal

- **Curso:** Investigación de Operaciones I  
- **Tema:** Sensibilidad (cambios en RHS y en coeficientes de la FO)  
- **Profesor:** Heriberto Felizzola
- **Herramientas:** Python + Gamspy

---
## Enunciado del ejemplo (JOBCO)

*Ejemplo – Taha (2010).* JOBCO fabrica dos productos en dos máquinas.  

- Una unidad del **producto 1** requiere **2 horas en la máquina 1** y **1 hora en la máquina 2**.  
- Una unidad del **producto 2** requiere **1 hora en la máquina 1** y **3 horas en la máquina 2**.  
- Los ingresos por unidad de los productos 1 y 2 son de **$30** y **$20**, respectivamente.  
- El tiempo de procesamiento **diario** total disponible en **cada** máquina es de **8 horas**.

Si $x_1$ y $x_2$ son las cantidades diarias (unidades) de los productos 1 y 2, respectivamente, el modelo de PL es:

\begin{align*}
\textbf{Maximizar } z &= 30 x_1 + 20 x_2 \\
\textbf{s.a.}& \\
            &2x_1 + x_2 \le 8 \quad \text{(Máquina 1)} \\
            &x_1 + 3x_2 \le 8 \quad \text{(Máquina 2)} \\
            &x_1, x_2 \ge 0\
\end{align*}





In [1]:
!pip install gamspy --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 5.1 MB/s eta 0:00:00


In [2]:
# Importar librerias
import numpy as np # Manejar vectores y matrices
import pandas as pd # Manejar tablas
import matplotlib.pyplot as plt # Generar gráficos
from gamspy import Container, Set, Parameter, Variable, Equation, Model, Sense, Sum, Options
import sys

## Caso 1. Cambios en el lado derecho

In [3]:
# Container
m = Container()

# Sets
productos = Set(m, name='productos', records=['p1', 'p2'])
maquinas = Set(m, name='maquinas', records=['m1', 'm2'])

# Variables
x = Variable(m, name='x', type='Positive', domain=[productos])

# Parametros
ingresos = Parameter(m, name='ingresos', domain=[productos], records=np.array([30,20]))
tiempo = Parameter(m, name='tiempo', domain=[maquinas,productos], records=np.array([[2,1],
                                                                                    [1,3]]))
capacidad = Parameter(m, name='capacidad', domain=[maquinas], records=np.array([8,8]))

# Restricción de capacidad por maquina
capacidad_max = Equation(m, name="capacidad_max", domain=[maquinas])
capacidad_max[...] = Sum(productos, tiempo[maquinas, productos] * x[productos]) <= capacidad[maquinas]

# Función objetivo
objetivo = Sum(productos, ingresos[productos] * x[productos])

# Crear y resolver el modelo
modelo = Model(m,
               name="Produccion",
               equations=[capacidad_max],
               sense=Sense.MAX,
               objective=objetivo,
               problem="LP")
modelo.solve(solver="cplex",
             solver_options={"objrng": "all", "rhsrng":"all"}, # Configura el análisis de sensibilidad
             output=sys.stdout)

--- Job _5jt1q0GVTLWPIwkujIyJLQ.gms Start 09/04/25 18:36:38 50.4.1 84b10359 LEX-LEG x86 64bit/Linux
--- Applying:
    /usr/local/lib/python3.12/dist-packages/gamspy_base/gmsprmun.txt
--- GAMS Parameters defined
    LP cplex
    Input /tmp/tmpc71zypkl/_5jt1q0GVTLWPIwkujIyJLQ.gms
    Output /tmp/tmpc71zypkl/_5jt1q0GVTLWPIwkujIyJLQ.lst
    ScrDir /tmp/tmpc71zypkl/tmp7lk75yn4/
    SysDir /usr/local/lib/python3.12/dist-packages/gamspy_base/
    LogOption 3
    Trace /tmp/tmpc71zypkl/_5jt1q0GVTLWPIwkujIyJLQ.txt
    License /usr/local/lib/python3.12/dist-packages/gamspy_base/gamslice.txt
    OptFile 1
    OptDir /tmp/tmpc71zypkl/
    LimRow 0
    LimCol 0
    TraceOpt 3
    GDX /tmp/tmpc71zypkl/_5jt1q0GVTLWPIwkujIyJLQout.gdx
    SolPrint 0
    SolveLink 2
    PreviousWork 1
    gdxSymbols newOrChanged
System information: 1 physical cores and 13 Gb physical memory detected
--- Starting compilation
--- _5jt1q0GVTLWPIwkujIyJLQ.gms(71) 3 Mb
--- Starting execution: elapsed 0:00:00.001
--- Generati

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,128.0,3,3,LP,CPLEX,0.01


In [4]:
x.records

,productos,level,marginal,lower,upper,scale
0,p1,3.2,0.0,0.0,inf,1.0
1,p2,1.6,0.0,0.0,inf,1.0


In [5]:
capacidad_max.records

,maquinas,level,marginal,lower,upper,scale
0,m1,8.0,14.0,-inf,8.0,1.0
1,m2,8.0,2.0,-inf,8.0,1.0
